In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from copy import deepcopy
#imdb = keras.datasets.imdb
#(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)


In [36]:
# Initialize ssome of the parameters
d = 5 #d
k = 6 #k

In [3]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))


In [39]:

def one_iter(examples, transforms, b, learning_rate=0.1):
    '''
    Runs one iteration through all provided examples
    '''
    total_loss = 0
    for q,h,t in examples:
        # project the query with all of the projection matrices
        
        p = [np.dot(transforms[i],q).T for i in range(k)]
        
        
        # Normalize the projections
        for i in range(k):
            p[i] = p[i] / np.sqrt((np.sum(p[i]**2)))
        
        # compute similarities between all projections and the candidate
        s = np.dot(p,h)
        
        # Apply sigmoid function and find the transformation that resulted
        # in the closest projection to the candidate
        sigmoids = sigmoid(np.add(s,b))
        maxsim = np.argmax(sigmoids)
        
        # y: predicted similarity
        # x: the corresponding projected vector 
        y = sigmoids[maxsim]
        x = p[maxsim]
        
        if(y == 0 or y==1):
            print("Perfect hit")
            continue
        # Compute the loss and update the corresponding projection matrix
        # in accordance with gradient descent.
        loss = t*np.log(y) + (np.subtract(1,t)*np.log(np.subtract(1,y)))
        total_loss += loss
        
        gradient = np.dot(x.T,loss)
        
        #print("Grad: ",gradient,"\n")
        #print("Grad*lr: ", np.multiply(learning_rate,gradient))
        prev_theta = transforms[maxsim]
        #print("Before: ", prev_theta)
        transforms[maxsim] = np.subtract(prev_theta, np.multiply(learning_rate,gradient))
        #print("OLD: ", prev_theta ,"\n", "NEW: ", transforms[maxsim],"\n")
        #print("After: ", transforms[maxsim],"\n")
        #Update the bias
        b_loss = y + (b[maxsim]-t)
        prev_b = b[maxsim]
        b[maxsim] =  np.subtract(prev_b, np.multiply(learning_rate, b_loss))
    return total_loss
        
def train(embeddings, train_examples, num_iter=1000):
    '''
    The main training algorithm. 
    '''
    transforms = [] #phi
    for i in range(k):
        transforms.append(np.identity(d) +np.random.normal(0, 0.1, [d,d]))
    b = np.zeros(k)  #bias
    
    for it in range(num_iter):
        loss = one_iter(train_examples, transforms,b)
        print("Loss, iter: ", loss/len(train_examples), it, "\n")
    #print(b)
    return (transforms, b)
    
def extract(q, transforms, embeddings, b, n=3):
    '''
    Extract the n top ranked candidates
    '''
    #print("Q: ", q,"\n")
    # p: All projections of q, list of k vectors
    p = [np.dot(transforms[i],q).T for i in range(k)]
    
    # Normalize all projections
    for i in range(k):
        p[i] = p[i] / np.sqrt((np.sum(p[i]**2)))
    
    # s: similarities of all projections of q, with all other terms
    s = [(np.dot(p,embeddings[h]), h) for h in embeddings]
    
    #print("Projections: ",p,"\n")
    #print("embeddings: ", embeddings,"\n")
    maxsims = [(_s[np.argmax(_s)], np.argmax(_s), h) for _s,h in s]
    #print(s)
    sigmoids = [(sigmoid(np.add(_s,b[idx])), h) for _s,idx,h in maxsims]
    sigmoids.sort()
    return sigmoids[-n:]

In [24]:
# Read in testing environment
em = False
test_embeddings = {}
train_examples = []
testfile = "/home/johannes/thesis_code/ml_experimentation/test_setup.txt"

fp = open(testfile,'r')
for line in fp:
    
    if("Embeddings" in line):
        #print(line)
        em = True
        continue
    if not line or line.startswith("#"):
        continue
    if em:
        emb = np.array([int(i) for i in line.split()[1:]])
        test_embeddings[line.split()[0]] = emb
    else:
        q,h,t = line.split() 
        train_examples.append((q,h,int(t)))

for w in test_embeddings:
    test_embeddings[w] = test_embeddings[w] / np.sqrt((np.sum(test_embeddings[w]**2)))
    
train_embeddings = [(test_embeddings[q],test_embeddings[h],t) for q,h,t in train_examples]
print(train_embeddings)
fp.close()

[(array([-0.18731716, -0.65561007,  0.56195149,  0.28097574,  0.37463432]), array([ 0.19900744,  0.59702231,  0.59702231, -0.39801488, -0.29851116]), 1), (array([-0.18731716, -0.65561007,  0.56195149,  0.28097574,  0.37463432]), array([ 0.44232587,  0.58976782, -0.58976782,  0.14744196,  0.29488391]), 1), (array([-0.18731716, -0.65561007,  0.56195149,  0.28097574,  0.37463432]), array([ 0.1796053 ,  0.53881591,  0.3592106 , -0.1796053 , -0.71842121]), 1), (array([-0.18731716, -0.65561007,  0.56195149,  0.28097574,  0.37463432]), array([ 0.91499142, -0.15249857, -0.15249857, -0.15249857, -0.30499714]), 0), (array([-0.18731716, -0.65561007,  0.56195149,  0.28097574,  0.37463432]), array([-0.85714286,  0.14285714, -0.28571429, -0.28571429,  0.28571429]), 0), (array([-0.18731716, -0.65561007,  0.56195149,  0.28097574,  0.37463432]), array([ 0.        ,  0.11470787, -0.57353933,  0.57353933, -0.57353933]), 0), (array([-0.18731716, -0.65561007,  0.56195149,  0.28097574,  0.37463432]), array(

In [41]:
transforms,b = train(test_embeddings, train_embeddings, 100)
print(b)

Loss, iter:  -0.7826396667626596 0 

Loss, iter:  -0.747866535752945 1 

Loss, iter:  -0.7336575310923354 2 

Loss, iter:  -0.7174338182067531 3 

Loss, iter:  -0.7100383615494041 4 

Loss, iter:  -0.7035941169625203 5 

Loss, iter:  -0.6947262642331737 6 

Loss, iter:  -0.689150373593784 7 

Loss, iter:  -0.6846491623421993 8 

Loss, iter:  -0.6813568258323931 9 

Loss, iter:  -0.679846285195199 10 

Loss, iter:  -0.6767688819620107 11 

Loss, iter:  -0.6732215168280995 12 

Loss, iter:  -0.6726837424067694 13 

Loss, iter:  -0.6708533945822608 14 

Loss, iter:  -0.6677654562682035 15 

Loss, iter:  -0.6691663757372718 16 

Loss, iter:  -0.6689909238993574 17 

Loss, iter:  -0.6700886586452846 18 

Loss, iter:  -0.6669218547791094 19 

Loss, iter:  -0.669165735888176 20 

Loss, iter:  -0.6654126828108695 21 

Loss, iter:  -0.6682461093173692 22 

Loss, iter:  -0.6658958308882367 23 

Loss, iter:  -0.6675609902951382 24 

Loss, iter:  -0.6655273691310768 25 

Loss, iter:  -0.6631333826

In [20]:
query = test_embeddings["volvo"]

predictions = extract(query, transforms, test_embeddings, b,15)
print(predictions)

[(0.28652179431894764, 'music'), (0.324133318600194, 'boi'), (0.42408666662429156, 'food'), (0.43308712770243246, 'fruit'), (0.43586818686745243, 'cat'), (0.46230734498396053, 'brand'), (0.4800638221862114, 'car'), (0.5027621490635844, 'vehichle'), (0.5048742696314579, 'metallica'), (0.5169823487957836, 'volvo'), (0.5257952339801082, 'band'), (0.5890236012185236, 'group'), (0.6767078147867991, 'horse'), (0.7013302634092692, 'pelle')]
